Testing for `gp_grid` regression models. Will raise error if fails.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy.testing import assert_array_almost_equal
import GPy
import gp_grid
from pdb import set_trace
from time import time
from gp_grid.misc import rastrigin, grid2mat
# gp_grid.debug()

# `gp_grid.models.GPGridRegression` Testing
Compare to a GPy model.

In [3]:
for n in [10, 20]: # number of points along each dimension
    print '\n' + '*' * 80
    print 'n = %d\n' % n
    
    # get the dataset
    np.random.seed(0)
    d = 2
    N = n**d
    # get the training set
    xg = [np.linspace(0,1,num=n).reshape((-1,1)),]*d
    x = grid2mat(*xg)
    y = rastrigin((x*2-1)*5.12, lin_term=1.)
    x = np.asarray(x).reshape((-1,d))
    y = np.asarray(y).reshape((-1,1))
    yg = y.reshape((n,)*d, order='F')
    
    # get testing set
    n_test = n-1
    xxg = [np.linspace(0,1,num=n_test).reshape((-1,1)),]*d
    xx = grid2mat(*xxg)
    yy = rastrigin((xx*2-1)*5.12, lin_term=1.)
    xx = np.asarray(xx).reshape((-1,d))
    yy = np.asarray(yy).reshape((-1,1))
    yyg = yy.reshape((n_test,)*d, order='F')

    # plot the training data
    # plt.figure()
    # plt.imshow(yg,interpolation='none')
    # plt.show()

    # get the gp_grid model
    kern_list = [gp_grid.kern.RBF(1) for i in range(d)]
    m_kml = gp_grid.models.GPGridRegression(xg,yg,kern_list)

    # get the GPy model
    kern = GPy.kern.RBF(d,ARD=True)
    m_gpy = GPy.models.GPRegression(x,y,kern)

    print "comparing log likelihood"
    assert_array_almost_equal(m_gpy.log_likelihood(), m_kml.log_likelihood(), decimal=2)

    print "checking gradient"
    m_kml.checkgrad()

    print 'testing if alpha matches'
    m_kml.fit()
    assert_array_almost_equal(m_gpy.posterior.woodbury_vector, m_kml._alpha, decimal=3)

    print 'testing if predictions match...'
    yyh_gpy = m_gpy.predict(xx,full_cov=True)
    yyh_kml = m_kml.predict_grid(xxg, compute_var='full')
    print '... mean'
    assert_array_almost_equal(yyh_gpy[0], yyh_kml[0].reshape((-1,1), order='F'), decimal=3)
    print '... variance'
    assert_array_almost_equal(yyh_gpy[1], yyh_kml[1], decimal=3)
        
    print "optimizing"
    t0 = time();    m_kml.optimize(factr=1e7);    t1 = time();    
    print 'gp_grid opt time = %g seconds. Log_like=%g' % (t1-t0, m_kml.log_likelihood())
    t0 = time();    m_gpy.optimize();    t1 = time();    
    print 'gpy opt time = %g seconds. Log_like=%g' % (t1-t0, m_gpy.log_likelihood())
    print "checking final log-likelihood"
    assert_array_almost_equal(m_gpy.log_likelihood(), m_kml.log_likelihood(), decimal=1)

print 'done tests.'


********************************************************************************
n = 10

[ 14:17:30 ] GP INFO: initializing Y
[ 14:17:30 ] GP INFO: initializing inference method
[ 14:17:30 ] GP INFO: adding kernel and likelihood as parameters
comparing log likelihood
checking gradient
[ 14:17:30 ] gp_grid.models INFO: Gradient check passed.
testing if alpha matches
testing if predictions match...
... mean
... variance
optimizing
[ 14:17:30 ] gp_grid.models INFO: Function Evals: 37. Log-Marginal Likelihood: -350.512.
gp_grid opt time = 0.297217 seconds. Log_like=-350.512
gpy opt time = 0.574087 seconds. Log_like=-350.508
checking final log-likelihood

********************************************************************************
n = 20

[ 14:17:31 ] GP INFO: initializing Y
[ 14:17:31 ] GP INFO: initializing inference method
[ 14:17:31 ] GP INFO: adding kernel and likelihood as parameters
comparing log likelihood
checking gradient
[ 14:17:31 ] gp_grid.models INFO: Gradient check passe

# `gp_grid.models.GPRegression` Testing
Will compare with GPy. 
Note that this assumes the model defaults are the same as those for GPy.

In [5]:
# generate data
np.random.seed(0)
N = 150
d = 2
x = np.random.uniform(size=(N,d)) # generate dataset
y = rastrigin((x*2-1)*5.12, lin_term=1.)
xx = np.random.uniform(size=(N+1,d)) # generate test set

# initialize GPy.models.GPRegression
kern = GPy.kern.RBF(d)
mg = GPy.models.GPRegression(x,y,kern)

# initialize gp_grid model
kern = gp_grid.kern.RBF(d)
m = gp_grid.models.GPRegression(x,y,kern)

print 'checking gradient'
m.checkgrad() 

print 'checking initial log-likelihood'
assert_array_almost_equal(mg.log_likelihood(), m.log_likelihood(), decimal=3) 

print 'testing if alpha matches'
m.fit()
alpha_gpy = mg.posterior.woodbury_vector
assert_array_almost_equal(alpha_gpy, m._alpha, decimal=3)

print 'testing if predictions match...'
yyh = m.predict(xx, compute_var='full')
yyh_gpy = mg.predict(xx,full_cov=True)
print '... mean'
assert_array_almost_equal(yyh_gpy[0], yyh[0], decimal=3)
print '... variance'
assert_array_almost_equal(yyh_gpy[1], yyh[1], decimal=2)

print 'checking final log-likelihood after optimizing'
m.optimize(factr=1e7)
mg.optimize()
ll_gpy_opt = mg.log_likelihood()
assert_array_almost_equal(ll_gpy_opt, m.log_likelihood(), decimal=2) 
print 'done tests.'

[ 14:18:03 ] GP INFO: initializing Y
[ 14:18:03 ] GP INFO: initializing inference method
[ 14:18:03 ] GP INFO: adding kernel and likelihood as parameters
checking gradient
[ 14:18:03 ] gp_grid.models INFO: Gradient check passed.
checking initial log-likelihood
testing if alpha matches
testing if predictions match...
... mean
... variance
checking final log-likelihood after optimizing
[ 14:18:04 ] gp_grid.models INFO: Function Evals: 74. Log-Marginal Likelihood: -564.128.
done tests.
